Note: Responses from local models can be quite slow, especially with 8-bit quantization.

With 4bit quantization, llama2-7b-chat uses about 8GB of VRAM

In [ ]:
!pip install llama-index transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Setup

### Data

In [ ]:
from llama_index.readers import BeautifulSoupWebReader

url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3149405/"

documents = BeautifulSoupWebReader().load_data([url])

In [ ]:
from llama_index.readers import BeautifulSoupWebReader

url1 = "https://www.researchgate.net/publication/359389896_Automated_Clinical_Coding_What_Why_and_Where_We_Are"

documents1 = BeautifulSoupWebReader().load_data([url1])


In [ ]:
documents1

[Document(id_='f74f9643-05c0-4ae1-907e-d2c2f461ae89', embedding=None, metadata={'URL': 'https://www.researchgate.net/publication/359389896_Automated_Clinical_Coding_What_Why_and_Where_We_Are'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='1561151030a9c98490151f292ce8b230c7fe20fef26582f673c9103bba1d378f', text='Just a moment...Enable JavaScript and cookies to continue', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')]

In [ ]:
documents

[Document(id_='fb233de5-c200-4a23-9ff3-02b9e598ee83', embedding=None, metadata={'URL': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3149405/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='1be03ac641343da4a422cf5a2eb31e6d50284f3c7fa79901ac6005a081e9c38d', text='\n\n\n\n\n\n\n\n\n\n\n\n\n\nPage not available - PMC\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main page content\n\n\n\n\n\n\nAn official website of the United States government\n\nHere\'s how you know\n\n\n\n\n\n\n\n\nThe .gov means it’s official.\n\n            Federal government websites often end in .gov or .mil. Before\n            sharing sensitive information, make sure you’re on a federal\n            government site.\n          \n\n\n\n\n\n\nThe site is secure.\n\n            The https:// ensures that you are connecting to the\n            official website and that any information you provide is encrypted\n            and transmitted securely.\n 

### LLM

This should run on a T4 GPU in the free tier

In [ ]:
# huggingface api token for downloading llama2
hf_token = ""

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Index Setup

In [ ]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents1, service_context=service_context)

In [ ]:
from llama_index import SummaryIndex

summary_index = SummaryIndex.from_documents(documents1, service_context=service_context)

### Helpful Imports / Logging

In [ ]:
from llama_index.response.notebook_utils import display_response

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Basic Query Engine

### Compact (default)

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What is the difference between Clinical, Manual and Automated coding?")

display_response(response)

**`Final Response:`** Based on the provided context information, the difference between Clinical, Manual, and Automated coding can be explained as follows:
Clinical coding refers to the process of assigning standardized codes to diagnoses, procedures, and other medical information to facilitate the electronic exchange and storage of healthcare data. Clinical coding is performed by trained healthcare professionals, such as medical record technicians, coders, and clinical documentation improvement specialists, who review medical records and assign the appropriate codes using manual methods, such as looking up code books or using software applications.
Manual coding, on the other hand, refers to the process of assigning codes to medical information without the use of automated systems or software. This method is often time-consuming and prone to errors, as it requires manual review of medical records and the use of code books or other reference materials.
Automated coding, also known as computer-assisted coding (CAC), refers to the use of software applications or other automated systems to assign codes to medical information. These systems use natural language processing (NLP) and machine learning algorithms to analyze medical records and assign codes based on patterns and rules learned from large datasets. Automated coding can be more efficient

### Refine

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("What is the difference between Clinical, Manual and Automated coding?")

display_response(response)

**`Final Response:`** Based on the provided context information, the difference between Clinical, Manual, and Automated coding can be understood as follows:
Clinical coding refers to the process of assigning standardized codes to diagnoses, procedures, and other medical information recorded in clinical settings. These codes are used to document and classify medical information for reimbursement, research, and quality improvement purposes. Clinical coding is typically performed by trained healthcare professionals, such as medical record technicians, coders, or clinical documentation specialists.
Manual coding, on the other hand, refers to the process of assigning codes to medical information using a manual system, without the use of automated tools or software. This method is often time-consuming and prone to errors, as it requires manual review and interpretation of medical records to identify and assign the appropriate codes.
Automated coding, as the name suggests, involves the use of software or computer algorithms to assign codes to medical information. This method is faster and more accurate than manual coding, as it can quickly and accurately identify and assign codes based on patterns and rules programmed into the software. Automated coding can also be more efficient and cost-effective than manual coding, as it

### Tree Summarize

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("What is the difference between Clinical, Manual and Automated coding?")

display_response(response)

**`Final Response:`** Based on the information provided in the article, the difference between clinical coding, manual coding, and automated coding can be summarized as follows:
Clinical coding refers to the process of assigning standardized codes to diagnoses, procedures, and other medical data to facilitate the electronic exchange and analysis of healthcare information. Clinical coding is performed by trained healthcare professionals, such as medical records technicians or coders, who review medical records and assign the appropriate codes using manual coding methods.
Manual coding, on the other hand, involves assigning codes to medical data using a manual process without the use of automated tools or software. This method is often time-consuming and prone to errors, as it requires a high level of accuracy and attention to detail.
Automated coding, also known as computer-assisted coding (CAC), uses software programs or algorithms to assign codes to medical data based on patterns and rules learned from large datasets. Automated coding can improve accuracy and efficiency compared to manual coding, as it can process large volumes of data quickly and accurately. However, automated coding may not always be able to capture the full complexity of medical data, and may require human review and editing to ensure accuracy.

## Router Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

### Single Selector

In [ ]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=False
)

response = query_engine.query("What is the difference between Clinical, Manual and Automated coding?")

display_response(response)

**`Final Response:`** Based on the provided context information, the difference between Clinical, Manual, and Automated coding can be summarized as follows:
Clinical coding refers to the process of assigning standardized codes to diagnoses, procedures, and other medical information to facilitate the electronic exchange and analysis of healthcare data. Clinical coding is typically performed by trained healthcare professionals, such as medical record technicians or coders, who review medical records and assign codes based on the information contained within them.
Manual coding, on the other hand, refers to the process of assigning codes to medical information without the use of automated coding systems. This method is often used in small healthcare facilities or in situations where automated coding systems are not available or practical.
Automated coding, as the name suggests, involves the use of computer algorithms and natural language processing techniques to assign codes to medical information without the need for human intervention. Automated coding systems use machine learning and other artificial intelligence techniques to analyze medical records and assign codes based on patterns and rules learned from large datasets.
In summary, the main difference between Clinical, Manual, and Automated coding is the level of human intervention involved in the process. Clinical coding

### Multi Selector

In [ ]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=True,
)

response = query_engine.query("What was mentioned about coding? Please summarize the results.")

display_response(response)

**`Final Response:`** Based on the information provided in the article "Automated Clinical Coding: What, Why, and Where We Are" on ResearchGate, the following points were mentioned about coding:
1. Automated clinical coding is the use of computer algorithms and natural language processing (NLP) techniques to extract clinical information from unstructured clinical text, such as clinical notes, discharge summaries, and radiology reports.
2. The goal of automated clinical coding is to convert unstructured clinical text into structured data that can be easily analyzed and used for various purposes, such as clinical decision support, research, and population health management.
3. Automated clinical coding can be used for a wide range of clinical specialties, including cardiology, oncology, and psychiatry.
4. Automated coding can improve the accuracy and efficiency of clinical coding, reduce the workload of coders, and provide real-time coding support.
5. There are several challenges associated with automated clinical coding, including the complexity of clinical language, the variability of clinical practices, and the need for high-quality training data.
In summary, the

## SubQuestion Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.query_engine import SubQuestionQueryEngine
import json

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    use_async=True,
    verbose=True,
)

query = "What was mentioned about coding? Please summarize the results."
# json_query = json.dumps(query)

response = query_engine.query(query)

display_response(response)

ValueError: ignored

## SQL Query Engine

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!curl https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O /content/chinook.zip
!unzip /content/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  1334k      0 --:--:-- --:--:-- --:--:-- 1338k
curl: (3) URL using bad/illegal format or missing URL
Archive:  /content/chinook.zip
  inflating: chinook.db              


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////content/chinook.db")

In [ ]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    service_context=service_context
)

In [ ]:
response = query_engine.query("What are some albums?")

display_response(response)

NotImplementedError: ignored

In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

display_response(response)

NotImplementedError: ignored

This last query should be a more complex join

In [ ]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 3")

display_response(response)

NotImplementedError: ignored

In [ ]:
print(response.metadata['sql_query'])

## Programs

Depending the LLM, you will have to test with either `OpenAIPydanticProgram` or `LLMTextCompletionProgram`

In [ ]:
from typing import List
from pydantic import BaseModel

from llama_index.program import OpenAIPydanticProgram, LLMTextCompletionProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Album),
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=True,
)

This seems to error out only because it ran out of output token space. We could fix this by setting `max_new_tokens` on the constructor higher than the default of 256.

In [ ]:
output = program(movie_name="The Shining")

ValidationError: ignored

In [ ]:
print(output)

## Data Agent

Similar to programs, OpenAI LLMs will use `OpenAIAgent`, while other LLMs will use `ReActAgent`.

In [ ]:
from llama_index.agent import OpenAIAgent, ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

In [ ]:
response = agent.chat("Hello!")
print(response)

Response: Great, I'm happy to help you with your question! Can you please provide more context or clarify what you need help with? For example, what kind of information are you trying to find or what question are you trying to answer?
Great, I'm happy to help you with your question! Can you please provide more context or clarify what you need help with? For example, what kind of information are you trying to find or what question are you trying to answer?


Interesting tool inputs and responses, but I guess it works lol

In [ ]:
response = agent.chat("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")
print(response)

Response: Great, let's get started! Based on your question, I understand that you want to know the difference between Meta and OpenAI. To answer this question, I will use the `summary` tool to summarize some relevant information about both Meta and OpenAI.
Action: summary
Action Input: {"text": "Meta and OpenAI are both AI research organizations, but they have some key differences. Meta was founded in 2015 by Mark Zuckerberg and is focused on developing AI technologies for Facebook and other Facebook-owned platforms. OpenAI, on the other hand, was founded in 2015 by a group of prominent AI researchers and is focused on developing AI technologies for a wide range of applications, including but not limited to natural language processing, computer vision, and robotics. Additionally, OpenAI is a non-profit organization, while Meta is a for-profit company. Both organizations have made significant contributions to the field of AI and have published numerous research papers and articles. Howe